Report for Task A based on Deep Neural Network using H2O and python.


This imports the h2o library and connects with the h2o server. The server has to be running.

In [4]:
import h2o

# switch to java 8 "sdk use java 8.0.452-tem"
# java -jar h2o.jar
h2o.init(ip="localhost", port="54321")

Checking whether there is an H2O instance running at http://localhost:54321. connected.
Attempting to start a local H2O server...
  Java Version: openjdk version "21.0.6" 2025-01-21 LTS; OpenJDK Runtime Environment Temurin-21.0.6+7 (build 21.0.6+7-LTS); OpenJDK 64-Bit Server VM Temurin-21.0.6+7 (build 21.0.6+7-LTS, mixed mode, sharing)
  Starting server from /home/adam/GithubProjects/pndc-lab/.venv/lib/python3.12/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp6tt5gale
  JVM stdout: /tmp/tmp6tt5gale/h2o_adam_started_from_python.out
  JVM stderr: /tmp/tmp6tt5gale/h2o_adam_started_from_python.err
  Server is running at http://127.0.0.1:54331
Connecting to H2O server at http://127.0.0.1:54331 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Europe/Warsaw
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,1 month and 26 days
H2O_cluster_name:,H2O_from_python_adam_j96fnj
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4.820 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


Helper function to build and test deep neural network model

In [12]:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

# def run_experiment(dataset, features, target, hidden, nfolds, activation):
#     train, test = dataset.split_frame(ratios=[0.8])
#     dnn = H2ODeepLearningEstimator(activation=activation, hidden=hidden, nfolds=nfolds)
#     dnn.train(x=features, y=target, training_frame=train, validation_frame=test)
#     print(dnn.show())
metrics = []

def run_experiment(dataset, features, target, hidden, nfolds, activation):
    train, test = dataset.split_frame(ratios=[0.8], seed=42)
    dnn = H2ODeepLearningEstimator(activation=activation, hidden=hidden, nfolds=nfolds, seed=42, reproducible=True)
    dnn.train(x=features, y=target, training_frame=train, validation_frame=test)
    print(dnn.default_threshold())
    #print(dnn.mean_per_class_error(xval=True))
    #print(dnn.run_time / 1000)
    print(dnn.mean_per_class_error(thresholds=None, train=True))
    print(dnn.show())
    metrics.append([activation, "None" if nfolds == 0 else f"{nfolds}-folds", hidden, 
                    f"{dnn.mean_per_class_error(train=True) * 100}%", 
                    f"{dnn.mean_per_class_error(valid=True) * 100}%", 
                    f"{dnn.mean_per_class_error(xval=True) * 100}%" if nfolds != 0 else "-", 
                    f"{dnn.run_time / 1000} s"])

Iris dataset

In [5]:
col_names = ["party", "handicapped-infants", "water-project-cost-sharing", "adoption-of-the-budget-resolution", "physician-fee-freeze", "el-salvador-aid", "religious-groups-in-schools", "anti-satellite-test-ban", "aid-to-nicaraguan-contras", "mx-missile", "immigration", "synfuels-corporation-cutback", "education-spending", "superfund-right-to-sue", "crime", "duty-free-exports", "export-administration-act-south-africa"]
votes = h2o.import_file(path="datasets/votes/house-votes-84.data", col_names=col_names)

features = col_names[1:]
target = "party"

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [5,5,], cross folds = 0, activation function = "rectifier"

In [13]:
run_experiment(dataset=votes, features=features, target=target, hidden=[5,5,5], nfolds=0, activation="rectifier")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%
0.30527271696447134
[[0.47264731686597217, 0.024566048837893484]]


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748030172222_64


Status of Neuron Layers: predicting party, 2-class classification, bernoulli distribution, CrossEntropy loss, 397 weights/biases, 16.1 KB, 3,380 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight             weight_rms           mean_bias                bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  ---------------------  ----------  ----------------------  -------------------  -----------------------  -------------------
    1        64       Input      0.0
    2        5        Rectifier  0.0        0.0   0.0   0.26967561226956605    0.4383763074874878     0.0         -0.0063685370201710615  0.181473970413208    0.5651113056292081       0.06373605132102966
    3        5        Rectifier  0.0        0.0   0.0   0.00854968486353755    0.0160413458943367     0.0         -0.06654038399457932    0.4950789213180542   0.9434190147898665       0.07188418507575989
    4        5        Rectifier  0.0        0.0   0.0   0.005699532681610435   0.008003383874893188   0.0         0.06221109122037888     0.40610623359680176  1.1627540761348671       0.6661279201507568
    5        2        Softmax               0.0   0.0   0.0015363607555627823  0.0003694383194670081  0.0         1.6811179116368293      1.0909318923950195   -1.3877787807814457e-17  0.16258132457733154

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.02484122938261853
RMSE: 0.1576110065402113
LogLoss: 0.09519415971222477
Mean Per-Class Error: 0.024566048837893498
AUC: 0.9920197116799058
AUCPR: 0.979104947040616
Gini: 0.9840394233598115

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.47264731686597217
            democrat    republican    Error    Rate
----------  ----------  ------------  -------  -----------
democrat    199         7             0.034    (7.0/206.0)
republican  2           130           0.0152   (2.0/132.0)
Total       201         137           0.0266   (9.0/338.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.472647     0.966543  66
max f2                       0.472647     0.977444  66
max f0point5                 0.713803     0.966258  59
max accuracy                 0.613505     0.973373  64
max precision                0.947205     0.990099  32
max recall                   0.0624061    1         89
max specificity              0.97684      0.995146  0
max absolute_mcc             0.472647     0.944893  66
max min_per_class_accuracy   0.613505     0.970874  64
max mean_per_class_accuracy  0.472647     0.975434  66
max tns                      0.97684      205       0
max fns                      0.97684      82        0
max fps                      3.33231e-06  206       237
max tps                      0.0624061    132       89
max tnr                      0.97684      0.995146  0
max fnr                      0.97684      0.621212  0
max fpr                      3.33231e-06  1         237
max tpr                      0.0624061    1         89

Gains/Lift Table: Avg response rate: 39.05 %, avg score: 39.11 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.150888                    0.97684            2.5104    2.5104      

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [20,20,20], cross folds = 0, activation function = "rectifier"


In [5]:
run_experiment(dataset=votes, features=features, target=target, hidden=[20,20,20], nfolds=0, activation="rectifier")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1746802299721_2


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 1,003 weights/biases, 17.2 KB, 1,160 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms              momentum    mean_weight            weight_rms           mean_bias              bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  --------------------  ----------  ---------------------  -------------------  ---------------------  --------------------
    1        4        Input      0.0
    2        20       Rectifier  0.0        0.0   0.0   0.00197044758242555    0.001000694464892149  0.0         0.046314876340329644   0.26595497131347656  0.5074913080223602     0.0397254079580307
    3        20       Rectifier  0.0        0.0   0.0   0.0032449281900335334  0.002042192965745926  0.0         0.013230080467183143   0.2217227816581726   1.001555163140654      0.02530832588672638
    4        20       Rectifier  0.0        0.0   0.0   0.10502976474708703    0.3024805784225464    0.0         0.008270140827435171   0.2174254059791565   0.9986306332673699     0.01212373748421669
    5        3        Softmax               0.0   0.0   0.12649297259195008    0.3245224952697754    0.0         -0.022376375366002323  1.1674742698669434   -0.003326534348147315  0.012769386172294617

ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.031060196744546506
RMSE: 0.17623903297665505
LogLoss: 0.12682079322907577
Mean Per-Class Error: 0.026064084170253465
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  -------
41             0                  0                 0          0 / 41
0              39                 2                 0.0487805  2 / 41
0              1                  33                0.0294118  1 / 34
41             40                 35                0.0258621  3 / 116

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.974138
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.04813494207255037
RMSE: 0.21939676860097637
LogLoss: 0.16669955592054675
Mean Per-Class Error: 0.05787037037037037
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  ------
9              0                  0                 0          0 / 9
0              8                  1                 0.111111   1 / 9
0              1                  15                0.0625     1 / 16
9              9                  16                0.0588235  2 / 34

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.941176
2    1
3    1

Scoring History: 
    timestamp            duration    training_speed    epochs    iterations    samples    training_rmse    training_logloss    training_r2    training_classification_error    training_auc    training_pr_auc    validation_rmse    validation_lo

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [100,100,100], cross folds = 0, activation function = "rectifier"

In [6]:
run_experiment(dataset=votes, features=features, target=target, hidden=[100,100,100], nfolds=0, activation="rectifier")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1746802299721_3


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 21,003 weights/biases, 254.4 KB, 1,160 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate             rate_rms              momentum    mean_weight             weight_rms           mean_bias               bias_rms
--  -------  -------  ---------  ---------  ----  ----  --------------------  --------------------  ----------  ----------------------  -------------------  ----------------------  ---------------------
    1        4        Input      0.0
    2        100      Rectifier  0.0        0.0   0.0   0.003975911785237258  0.00319715216755867   0.0         0.004654598386259749    0.136976420879364    0.4954202504278193      0.007798504084348679
    3        100      Rectifier  0.0        0.0   0.0   0.008445915107091423  0.019364923238754272  0.0         -0.002800815973335193   0.1003444492816925   0.9927678236510526      0.007100751623511314
    4        100      Rectifier  0.0        0.0   0.0   0.07848945280713088   0.24993687868118286   0.0         -0.0005843455062503381  0.10042539238929749  0.9995317481906626      0.0019305003806948662
    5        3        Softmax               0.0   0.0   0.08357639898662456   0.25366687774658203   0.0         -0.012201107405126095   0.537407636642456    -0.0005347984026380365  0.0009671554435044527

ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.09215881650519689
RMSE: 0.3035767061307519
LogLoss: 0.33867796315886844
Mean Per-Class Error: 0.11666666666666665
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error    Rate
-------------  -----------------  ----------------  -------  --------
35             0                  0                 0        0 / 35
0              26                 14                0.35     14 / 40
0              0                  41                0        0 / 41
35             26                 55                0.12069  14 / 116

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.87931
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.06578954574795269
RMSE: 0.25649472849934496
LogLoss: 0.24157352864713874
Mean Per-Class Error: 0.06666666666666667
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  ------
15             0                  0                 0          0 / 15
0              8                  2                 0.2        2 / 10
0              0                  9                 0          0 / 9
15             8                  11                0.0588235  2 / 34

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.941176
2    1
3    1

Scoring History: 
    timestamp            duration    training_speed    epochs    iterations    samples    training_rmse    training_logloss    training_r2    training_classification_error    training_auc    training_pr_auc    validation_rmse    validation

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [5,5,5], cross folds = 5, activation function = "rectifier"


In [7]:
run_experiment(dataset=votes, features=features, target=target, hidden=[5,5,5], nfolds=5, activation="rectifier")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1746802299721_4


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 103 weights/biases, 6.1 KB, 1,356 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight           weight_rms           mean_bias            bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  ---------------------  ----------  --------------------  -------------------  -------------------  -------------------
    1        4        Input      0.0
    2        5        Rectifier  0.0        0.0   0.0   0.0020633155247196553  0.0006992283742874861  0.0         0.07138321688398719   0.45854246616363525  0.5134669413874761   0.07212996482849121
    3        5        Rectifier  0.0        0.0   0.0   0.002894400132354349   0.0025081196799874306  0.0         0.1764200609177351    0.4150749444961548   0.9950174868471026   0.05572769045829773
    4        5        Rectifier  0.0        0.0   0.0   0.2016345627361443     0.4037895202636719     0.0         -0.11414474956691265  0.4243086576461792   0.9515299331110117   0.04271313548088074
    5        3        Softmax               0.0   0.0   0.20418421717671056    0.40277421474456787    0.0         0.6632805724938711    1.6805429458618164   0.13688108591847745  0.5106709003448486

ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.15103512550177015
RMSE: 0.3886323783497332
LogLoss: 0.48103132997112186
Mean Per-Class Error: 0.16004273504273506
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error     Rate
-------------  -----------------  ----------------  --------  --------
43             0                  0                 0         0 / 43
0              31                 8                 0.205128  8 / 39
0              11                 29                0.275     11 / 40
43             42                 37                0.155738  19 / 122

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.844262
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.15616610499055758
RMSE: 0.39517857354689356
LogLoss: 0.49273846998789306
Mean Per-Class Error: 0.16363636363636366
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  ------
7              0                  0                 0          0 / 7
0              10                 1                 0.0909091  1 / 11
0              4                  6                 0.4        4 / 10
7              14                 7                 0.178571   5 / 28

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.821429
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on cross-validation data. **

MSE: 0.2099801128991822
RMSE: 0.4582358703759257
LogLoss: 0.6090748321718017
Mean Per-Class Error: 0.19242695289206915
AUC table was not computed: it is either disabled (model parameter 'auc_type' was s

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [20,20,20], cross folds = 5, activation function = "rectifier"


In [8]:
run_experiment(dataset=votes, features=features, target=target, hidden=[20,20,20], nfolds=5, activation="rectifier")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1746802299721_5


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 1,003 weights/biases, 17.2 KB, 1,378 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight           weight_rms           mean_bias               bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  ---------------------  ----------  --------------------  -------------------  ----------------------  --------------------
    1        4        Input      0.0
    2        20       Rectifier  0.0        0.0   0.0   0.002558920373849105   0.0015626857057213783  0.0         0.016038695781026034  0.29683637619018555  0.5333943166674966      0.07079270482063293
    3        20       Rectifier  0.0        0.0   0.0   0.0034413395349110944  0.002702178433537483   0.0         0.014085894276213366  0.21766537427902222  0.9905313879511087      0.04863758385181427
    4        20       Rectifier  0.0        0.0   0.0   0.0029846735174578498  0.004151705652475357   0.0         0.009239994478703011  0.22843873500823975  0.995465752420486       0.028229154646396637
    5        3        Softmax               0.0   0.0   0.01065075428923592    0.023494809865951538   0.0         0.028032822906970976  1.1358981132507324   -0.0023321860667721663  0.02531728893518448

ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.053516173195500054
RMSE: 0.23133562889338957
LogLoss: 0.17209550858103495
Mean Per-Class Error: 0.07142857142857142
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  -------
42             0                  0                 0          0 / 42
0              37                 0                 0          0 / 37
0              9                  33                0.214286   9 / 42
42             46                 33                0.0743802  9 / 121

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.92562
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.05108440233536721
RMSE: 0.22601858847308823
LogLoss: 0.15084120980313756
Mean Per-Class Error: 0.125
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error     Rate
-------------  -----------------  ----------------  --------  ------
8              0                  0                 0         0 / 8
0              13                 0                 0         0 / 13
0              3                  5                 0.375     3 / 8
8              16                 5                 0.103448  3 / 29

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.896552
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on cross-validation data. **

MSE: 0.07359897578301026
RMSE: 0.27129131166148734
LogLoss: 0.23371753214530006
Mean Per-Class Error: 0.10424710424710425
AUC table was not computed: it is either disabled (model parameter 'auc_t

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [100,100,100], cross folds = 5, activation function = "rectifier"

In [9]:
run_experiment(dataset=votes, features=features, target=target, hidden=[100,100,100], nfolds=5, activation="rectifier")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1746802299721_6


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 21,003 weights/biases, 254.4 KB, 1,330 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate             rate_rms               momentum    mean_weight             weight_rms           mean_bias               bias_rms
--  -------  -------  ---------  ---------  ----  ----  --------------------  ---------------------  ----------  ----------------------  -------------------  ----------------------  --------------------
    1        4        Input      0.0
    2        100      Rectifier  0.0        0.0   0.0   0.002604135101137217  0.0016099503263831139  0.0         -0.00499676844352507    0.13934433460235596  0.49538786735541224     0.013908583670854568
    3        100      Rectifier  0.0        0.0   0.0   0.005291094798070844  0.004142940044403076   0.0         -0.004732171249717884   0.10112830996513367  0.9920812843243936      0.007478686049580574
    4        100      Rectifier  0.0        0.0   0.0   0.028521074834337922  0.14549881219863892    0.0         0.00039673851135539737  0.10092613101005554  0.9995270312820571      0.003314962610602379
    5        3        Softmax               0.0   0.0   0.03679192417184822   0.1768796443939209     0.0         -0.015297418599560237   0.5598890781402588   -0.0004109971850362049  0.001897687092423439

ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.024918665286832432
RMSE: 0.15785647052570392
LogLoss: 0.07088674622441893
Mean Per-Class Error: 0.03968253968253968
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  -------
38             0                  0                 0          0 / 38
0              40                 2                 0.047619   2 / 42
0              3                  39                0.0714286  3 / 42
38             43                 41                0.0409836  5 / 122

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.959016
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.027046909927153865
RMSE: 0.1644594476676663
LogLoss: 0.07627798554620778
Mean Per-Class Error: 0.041666666666666664
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  ------
12             0                  0                 0          0 / 12
0              8                  0                 0          0 / 8
0              1                  7                 0.125      1 / 8
12             9                  7                 0.0357143  1 / 28

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.964286
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on cross-validation data. **

MSE: 0.11425305185713283
RMSE: 0.3380133900559752
LogLoss: 0.4131799315563886
Mean Per-Class Error: 0.15162907268170425
AUC table was not computed: it is either d

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [5,5,5], cross folds = 10, activation function = "rectifier"


In [10]:
run_experiment(dataset=votes, features=features, target=target, hidden=[5,5,5], nfolds=10, activation="rectifier")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1746802299721_7


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 103 weights/biases, 6.1 KB, 1,219 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight           weight_rms           mean_bias             bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  ---------------------  ----------  --------------------  -------------------  --------------------  -------------------
    1        4        Input      0.0
    2        5        Rectifier  0.0        0.0   0.0   0.0023835677056922577  0.0016095936298370361  0.0         0.02976214773952961   0.41344428062438965  0.4875544544520142    0.0789962112903595
    3        5        Rectifier  0.0        0.0   0.0   0.0023137907427735627  0.0016503781080245972  0.0         0.10635395685210824   0.3698540925979614   1.0545374485065844    0.06249949336051941
    4        5        Rectifier  0.0        0.0   0.0   0.2017270764982095     0.4037531614303589     0.0         -0.1411936704814434   0.47323012351989746  0.9868456509190662    0.059576615691185
    5        3        Softmax               0.0   0.0   0.20288692485773935    0.4032881259918213     0.0         -0.18530485431353252  2.0838003158569336   -0.08186995859757314  0.2853379249572754

ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.10691528748313972
RMSE: 0.32697903217659036
LogLoss: 0.34940072731816546
Mean Per-Class Error: 0.09252136752136753
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  --------
36             0                  0                 0          0 / 36
3              35                 1                 0.102564   4 / 39
0              7                  33                0.175      7 / 40
39             42                 34                0.0956522  11 / 115

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.904348
2    0.982609
3    1

ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.09358451708994402
RMSE: 0.30591586603173104
LogLoss: 0.30992442548247984
Mean Per-Class Error: 0.09393939393939395
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  ------
14             0                  0                 0          0 / 14
0              9                  2                 0.181818   2 / 11
0              1                  9                 0.1        1 / 10
14             10                 11                0.0857143  3 / 35

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.914286
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on cross-validation data. **

MSE: 0.09323511905129393
RMSE: 0.30534426317075936
LogLoss: 0.3107786910796217
Mean Per-Class Error: 0.10961538461538461
AUC table was not computed: it is either disabled (model paramet

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [20,20,20], cross folds = 10, activation function = "rectifier"

In [11]:
run_experiment(dataset=votes, features=features, target=target, hidden=[20,20,20], nfolds=10, activation="rectifier")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1746802299721_8


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 1,003 weights/biases, 17.2 KB, 1,303 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight            weight_rms           mean_bias             bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  ---------------------  ----------  ---------------------  -------------------  --------------------  --------------------
    1        4        Input      0.0
    2        20       Rectifier  0.0        0.0   0.0   0.0024943113872723187  0.0015386850573122501  0.0         -0.023311711894348264  0.2911614179611206   0.5152625058082638    0.09179544448852539
    3        20       Rectifier  0.0        0.0   0.0   0.004271377880213549   0.004004022106528282   0.0         0.0048572227275144545  0.22880899906158447  1.0072660931411024    0.07085052132606506
    4        20       Rectifier  0.0        0.0   0.0   0.002495752225004253   0.004275936633348465   0.0         -0.009837898116529686  0.22042596340179443  0.9944462095139037    0.035468652844429016
    5        3        Softmax               0.0   0.0   0.03536717688063315    0.1102403998374939     0.0         -0.021939748308310907  1.1055517196655273   -0.06660811360574034  0.10556313395500183

ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.016622798853041514
RMSE: 0.12892943361793502
LogLoss: 0.06380174824404826
Mean Per-Class Error: 0.02439024390243903
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  -------
39             0                  0                 0          0 / 39
0              40                 1                 0.0243902  1 / 41
0              2                  39                0.0487805  2 / 41
39             42                 40                0.0247934  3 / 121

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.975207
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.003919787843565137
RMSE: 0.06260820907489
LogLoss: 0.028564655429938704
Mean Per-Class Error: 0.0
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error    Rate
-------------  -----------------  ----------------  -------  ------
11             0                  0                 0        0 / 11
0              9                  0                 0        0 / 9
0              0                  9                 0        0 / 9
11             9                  9                 0        0 / 29

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    1
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on cross-validation data. **

MSE: 0.03446672876556713
RMSE: 0.18565217145395077
LogLoss: 0.11624695739284671
Mean Per-Class Error: 0.04878048780487806
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO 

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [100,100,100], cross folds = 10, activation function = "rectifier"


In [12]:
run_experiment(dataset=votes, features=features, target=target, hidden=[100,100,100], nfolds=10, activation="rectifier")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1746802299721_9


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 21,003 weights/biases, 254.4 KB, 1,293 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight             weight_rms           mean_bias              bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  ---------------------  ----------  ----------------------  -------------------  ---------------------  ---------------------
    1        4        Input      0.0
    2        100      Rectifier  0.0        0.0   0.0   0.0034620113160053734  0.0032848818227648735  0.0         -0.0028028148018347564  0.13918203115463257  0.4952037036185499     0.01191757619380951
    3        100      Rectifier  0.0        0.0   0.0   0.010445187028840883   0.010375745594501495   0.0         -0.0039378229479631955  0.10092031955718994  0.992533769419278      0.010452166199684143
    4        100      Rectifier  0.0        0.0   0.0   0.06357169062291505    0.22021335363388062    0.0         2.6370294325397482e-05  0.1003589928150177   0.9998650832013364     0.003502568230032921
    5        3        Softmax               0.0   0.0   0.07745458227630782    0.23105275630950928    0.0         0.04368542918624977     0.5379626750946045   -0.001054557400315064  0.0032990165054798126

ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.0216280931733419
RMSE: 0.1470649284273511
LogLoss: 0.06445583521654032
Mean Per-Class Error: 0.032818532818532815
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error      Rate
-------------  -----------------  ----------------  ---------  -------
36             0                  0                 0          0 / 36
0              39                 3                 0.0714286  3 / 42
0              1                  36                0.027027   1 / 37
36             40                 39                0.0347826  4 / 115

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.965217
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.0007485156527967257
RMSE: 0.027359014104984224
LogLoss: 0.007823170192496953
Mean Per-Class Error: 0.0
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error    Rate
-------------  -----------------  ----------------  -------  ------
14             0                  0                 0        0 / 14
0              8                  0                 0        0 / 8
0              0                  13                0        0 / 13
14             8                  13                0        0 / 35

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    1
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on cross-validation data. **

MSE: 0.14493907908977055
RMSE: 0.3807086538151852
LogLoss: 0.7161126236275075
Mean Per-Class Error: 0.16902616902616904
AUC table was not computed: it is either disabled (model parameter 'auc_ty

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [5,5,5], cross folds = 0, activation function = "tanh"


In [13]:
run_experiment(dataset=votes, features=features, target=target, hidden=[5,5,5], nfolds=0, activation="tanh")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1746802299721_10


Status of Neuron Layers: predicting class, 3-class classification, multinomial distribution, CrossEntropy loss, 103 weights/biases, 6.1 KB, 1,240 training samples, mini-batch size 1
    layer    units    type     dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight           weight_rms          mean_bias               bias_rms
--  -------  -------  -------  ---------  ----  ----  ---------------------  ---------------------  ----------  --------------------  ------------------  ----------------------  --------------------
    1        4        Input    0.0
    2        5        Tanh     0.0        0.0   0.0   0.0023227178084198386  0.001781489234417677   0.0         0.08389420956373214   0.4738345146179199  -0.06185200833032976    0.07142803072929382
    3        5        Tanh     0.0        0.0   0.0   0.0021614224114455283  0.0011797240003943443  0.0         -0.09153357880190015  0.4301849603652954  0.03211802149238687     0.06830701231956482
    4        5        Tanh     0.0        0.0   0.0   0.001974017512984574   0.0012832293286919594  0.0         0.08565864361822605   0.4363778829574585  0.037335374615913054    0.032962679862976074
    5        3        Softmax             0.0   0.0   0.0069700710165003935  0.005926892161369324   0.0         0.6222223520278931    2.183352470397949   -0.0008344093526480451  0.016140557825565338

ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.0829068444888666
RMSE: 0.28793548667864227
LogLoss: 0.27845704188361636
Mean Per-Class Error: 0.11024505210551722
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error     Rate
-------------  -----------------  ----------------  --------  --------
38             1                  0                 0.025641  1 / 39
0              35                 8                 0.186047  8 / 43
0              5                  37                0.119048  5 / 42
38             41                 45                0.112903  14 / 124

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.887097
2    1
3    1

ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.10673356546625022
RMSE: 0.32670103376979115
LogLoss: 0.3137962392928355
Mean Per-Class Error: 0.2619047619047619
AUC table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).
AUCPR table was not computed: it is either disabled (model parameter 'auc_type' was set to AUTO or NONE) or the domain size exceeds the limit (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
Iris-setosa    Iris-versicolor    Iris-virginica    Error     Rate
-------------  -----------------  ----------------  --------  ------
11             0                  0                 0         0 / 11
0              5                  2                 0.285714  2 / 7
0              4                  4                 0.5       4 / 8
11             9                  6                 0.230769  6 / 26

Top-3 Hit Ratios: 
k    hit_ratio
---  -----------
1    0.769231
2    1
3    1

Scoring History: 
    timestamp            duration    training_speed    epochs    iterations    samples    training_rmse    training_logloss    training_r2    training_classification_error    training_auc    training_pr_auc    validation_rmse    validation_logloss    validation_r2    val

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [5,5,5], cross folds = 0, activation function = "tanh"


In [ ]:
run_experiment(dataset=votes, features=features, target=target, hidden=[5,5,5], nfolds=0, activation="tanh")


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [20,20,20], cross folds = 0, activation function = "tanh"


In [ ]:
run_experiment(dataset=votes, features=features, target=target, hidden=[20,20,20], nfolds=0, activation="tanh")


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [100,100,100], cross folds = 0, activation function = "tanh"


In [ ]:
run_experiment(dataset=votes, features=features, target=target, hidden=[100,100,100], nfolds=0, activation="tanh")

Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [5,5,5], cross folds = 5, activation function = "tanh"


In [ ]:
run_experiment(dataset=votes, features=features, target=target, hidden=[5,5,5], nfolds=5, activation="tanh")

Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [20,20,20], cross folds = 5, activation function = "tanh"


In [ ]:
run_experiment(dataset=votes, features=features, target=target, hidden=[20,20,20], nfolds=5, activation="tanh")

Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [100,100,100], cross folds = 5, activation function = "tanh"

In [ ]:
run_experiment(dataset=votes, features=features, target=target, hidden=[100,100,100], nfolds=5, activation="tanh")


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [5,5,5], cross folds = 10, activation function = "tanh"

In [ ]:
run_experiment(dataset=votes, features=features, target=target, hidden=[5,5,5], nfolds=10, activation="tanh")

Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [20,20,20], cross folds = 10, activation function = "tanh"


In [ ]:
run_experiment(dataset=votes, features=features, target=target, hidden=[20,20,20], nfolds=10, activation="tanh")

Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [20,20,20], cross folds = 10, activation function = "tanh"


In [ ]:
run_experiment(dataset=votes, features=features, target=target, hidden=[100,100,100], nfolds=10, activation="tanh")
